In [1]:
import joblib
import json
import pandas as pd
import numpy as np
import utils
from transformers import AutoTokenizer
import torch
import sys
from collections import Counter

sys.path.append("../datasets")
import configs

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
dataset = "dbpedia"
architecture = "BART"
data_dir = f"../datasets/{dataset}_dataset"
batch_size = 256
model_name = f"{dataset}_model_0.9_0.9_0.9"

In [3]:
if architecture == "BART":
    tokenizer = AutoTokenizer.from_pretrained("ModelTC/bart-base-mnli")
elif architecture == "ELECTRA":
    tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator")
else:
    print(f"Invalid backbone architecture: {architecture}")

In [4]:
all_datasets = utils.load_dataset(
    data_dir=data_dir,
    tokenizer=tokenizer,
    max_length=configs.dataset_to_max_length[dataset],
)

all_dataloaders = {
    dataset_name: torch.utils.data.DataLoader(
        all_datasets[dataset_name],
        batch_size=batch_size,
        shuffle=False,
        collate_fn=lambda batch: {
            "input_ids": torch.LongTensor([i["input_ids"] for i in batch]),
            "attention_mask": torch.Tensor([i["attention_mask"] for i in batch]),
            "label": torch.LongTensor([i["label"] for i in batch]),
        },
    )
    for dataset_name in all_datasets.keys()
    if dataset_name != "train"
}

Train data shape:  (24094, 2)


Map:   0%|          | 0/24094 [00:00<?, ? examples/s]

Map:   0%|          | 0/60794 [00:00<?, ? examples/s]

Map:   0%|          | 0/1836 [00:00<?, ? examples/s]

Map:   0%|          | 0/1281 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
bestk_train_data_per_proto = joblib.load(
    f"artifacts/{dataset}/{model_name}/bestk_train_data_per_proto.joblib"
)

best_protos_per_traineg = joblib.load(
    f"artifacts/{dataset}/{model_name}/best_protos_per_traineg.joblib"
)

In [6]:
num_protototypes = bestk_train_data_per_proto.indices.shape[1]
num_training_examples = best_protos_per_traineg[0].shape[0]

In [8]:
num_training_examples, all_datasets["test"].__len__()

(60794, 60794)

In [9]:
sorted(
    dict(Counter(all_datasets["test"]["label"])).items(),
    key=lambda x: x[1],
    reverse=True,
)

[(0, 31495),
 (2, 11574),
 (3, 5418),
 (1, 5300),
 (5, 4820),
 (6, 1483),
 (4, 444),
 (8, 198),
 (7, 62)]

In [11]:
indices = bestk_train_data_per_proto.indices.reshape(num_protototypes, -1).tolist()
values = bestk_train_data_per_proto.values.reshape(num_protototypes, -1).tolist()

indices_values_for_all_prototypes = []
for i in range(num_protototypes):
    indices_values_per_prototype = list(zip(indices[i], values[i]))
    indices_values_per_prototype.sort(key=lambda x: x[1], reverse=False)
    indices_values_for_all_prototypes.append(indices_values_per_prototype)

for i in range(num_protototypes):
    print(f"Prototype {i}")
    training_examples_indices = np.array(
        [x[0] for x in indices_values_for_all_prototypes[i]]
    )
    training_examples_distances = np.array(
        [x[1] for x in indices_values_for_all_prototypes[i]]
    )
    print(np.array(all_datasets["test"]["label"])[training_examples_indices])
    print(training_examples_distances)
    print("------------------")

Prototype 0
[0 0 0 0 2]
[-3.86000395 -3.8521657  -3.84659767 -3.8372066  -3.80397987]
------------------
Prototype 1
[1 0 3 0 0]
[-3.84467077 -3.83111954 -3.79959321 -3.7980237  -3.76843357]
------------------
Prototype 2
[0 5 0 6 0]
[-3.84857941 -3.84825969 -3.83848882 -3.80562568 -3.41503525]
------------------
Prototype 3
[0 5 0 1 2]
[-3.85732651 -3.84428358 -3.84127998 -3.83616781 -3.80349612]
------------------
Prototype 4
[0 2 1 0 0]
[-3.84601855 -3.84326577 -3.82910824 -3.79504251 -3.78783989]
------------------
Prototype 5
[0 0 0 3 1]
[-3.84707737 -3.84617877 -3.83714962 -3.76204228 -3.41468787]
------------------
Prototype 6
[5 0 5 2 3]
[-3.85466504 -3.8411448  -3.83814526 -3.83546615 -3.79589629]
------------------
Prototype 7
[2 3 0 5 3]
[-3.84377384 -3.83243132 -3.8015604  -3.79333043 -3.78693724]
------------------
Prototype 8
[3 0 3 0 4]
[-3.84385467 -3.83619356 -3.82836747 -3.75422311 -3.41041017]
------------------
Prototype 9
[0 3 2 2 1]
[-3.84332275 -3.83947897 -3.837

In [9]:
best_protos_per_testeg = joblib.load(
    f"artifacts/{dataset}/{model_name}/best_protos_per_testeg.joblib"
)

In [10]:
best_protos_per_testeg.keys()

dict_keys(['test_textfooler', 'test_textbugger', 'adv_textfooler', 'adv_textbugger', 'test_textfooler_protocnn', 'adv_textfooler_protocnn', 'test_textbugger_protocnn', 'adv_textbugger_protocnn', 'adv_paraphrased', 'test_paraphrased'])

In [12]:
test_dataset = "test_textbugger"
num_examples = len(all_datasets[test_dataset])


indices = best_protos_per_testeg[test_dataset][0].tolist()
values = best_protos_per_testeg[test_dataset][1].tolist()

prototoypes_indices_values_per_point = []
for i in range(num_examples):
    indices_values_per_training_example = list(zip(indices[i], values[i]))
    indices_values_per_training_example.sort(key=lambda x: x[1], reverse=False)
    prototoypes_indices_values_per_point.append(indices_values_per_training_example)

In [13]:
prototoypes_indices_values_per_training_example[0]

[(15, -3.8186159133911133),
 (10, -0.07565689831972122),
 (1, 0.017503520473837852),
 (5, 0.08167710900306702),
 (9, 0.1278260201215744)]